<a href="https://colab.research.google.com/github/Abhilaksh1498/SoC-20-Chatbot/blob/master/Pre_trained_vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


1.   We'll use 2 pre-trained word embeddings (GloVe, wiki-news) for Twitter Sentiment Analysis
2.   We'll also use Word2vec to train the embeddings specifically

In [0]:
import tensorflow as tf


**To add files from google drive**


```
from google.colab import drive
drive.mount('/content/gdrive')
```



*Importing a Kaggle dataset*
https://www.kaggle.com/general/74235

---

The second part of actually downloading dataset is mentioned here
https://towardsdatascience.com/setting-up-kaggle-in-google-colab-ebb281b61463

In [4]:
! pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"abhilakshmaheshwari","key":"b4131928857e3bb171320644901f7a27"}'}

In [0]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [0]:
! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets list

In [0]:
! mkdir kaggle_dataset

In [0]:
!kaggle datasets list -s sentiment # you can search for a specific dataset using this

ref                                                      title                                             size  lastUpdated          downloadCount  
-------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  
crowdflower/twitter-airline-sentiment                    Twitter US Airline Sentiment                       3MB  2019-10-16 00:04:05          39246  
harriken/emoji-sentiment                                 Emoji sentiment                                   12MB  2017-10-01 09:56:54           2381  
kazanova/sentiment140                                    Sentiment140 dataset with 1.6 million tweets      81MB  2017-09-13 22:43:19          29619  
bittlingmayer/amazonreviews                              Amazon Reviews for Sentiment Analysis            493MB  2019-11-18 02:50:34          24445  
arkhoshghalb/twitter-sentiment-analysis-hatred-speech    Twitter Sentiment Analysis                 

In [8]:
! kaggle datasets download -d kazanova/sentiment140 -p /content/kaggle_dataset 
# the download link was copied using get api from 3 dot menu on kaggle dataset page itself

 90% 73.0M/80.9M [00:01<00:00, 35.7MB/s]
100% 80.9M/80.9M [00:01<00:00, 44.2MB/s]


In [9]:
# to unzip file to a specific directory use this
! unzip /content/kaggle_dataset/sentiment140.zip -d /content/kaggle_dataset/twitter_sentiment 
# you can directly rename the file after that

Archive:  /content/kaggle_dataset/sentiment140.zip
  inflating: /content/kaggle_dataset/twitter_sentiment/training.1600000.processed.noemoticon.csv  


In [0]:
# loading the dataset
import pandas as pd
dataset = pd.read_csv(r'/content/kaggle_dataset/twitter_sentiment/twitter_sentiment.csv', encoding= 'latin1', header=None)

In [0]:
dataset.iloc[2:4,:]
# target variable is 0th index, text is last index
import numpy as np
X = dataset.iloc[:,-1].values
y = dataset.iloc[:,0].values

In [13]:
print(dataset.isnull().sum())
set(y)

0    0
1    0
2    0
3    0
4    0
5    0
dtype: int64


{0, 4}

In [0]:
# label encoding the dependent variable
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
# print(set(y))

In [0]:
# now we will clean the text and then tokenise using keras tokenizer
def clean_text(text):
       text = text.lower()
       text = re.sub(r"i'm", "i am", text)
       text = re.sub(r"he's", "he is", text)
       text = re.sub(r"she's", "she is", text)
       text = re.sub(r"that's", "that is", text)
       text = re.sub(r"what's", "what is", text)
       text = re.sub(r"where's", "where is", text)
       text = re.sub(r"how's", "how is", text)
       text = re.sub(r"\'ll", " will", text)
       text = re.sub(r"\'ve", " have", text)
       text = re.sub(r"\'re", " are", text)
       text = re.sub(r"\'d", " would", text)
       text = re.sub(r"won't", "will not", text)
       text = re.sub(r"can't", "cannot", text)
       text = re.sub(r"n't", " not", text)
       text = re.sub( '[^a-zA-Z]', ' ', text )
       return text

In [0]:
import re
for i in range(len(X)):
  X[i] = clean_text(X[i])

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
oov_token = '<OOV>'
trunc_type='post'
padding_type='post'
tokenizer = Tokenizer(oov_token = oov_token)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [0]:
# np.array([len(x) for x in X_train]).mean() = 14
max_length = 20
X_train = tf.keras.preprocessing.sequence.pad_sequences( X_train , maxlen=max_length , padding='post', truncating = trunc_type )
X_test = tf.keras.preprocessing.sequence.pad_sequences( X_test , maxlen=max_length , padding='post', truncating = trunc_type )

In [0]:
# set([len(x) for x in X_train]) = {20}
word_index = tokenizer.word_index
vocab_size = len(word_index)+1

Glove vectors link (50d, 100d, 200d):-

```
https://www.kaggle.com/rtatman/glove-global-vectors-for-word-representation#glove.6B.50d.txt
```



Glove vectors link (300d, 500d):-

```
https://nlp.stanford.edu/projects/glove/
```

Wiki-news vectors link :-


```
https://fasttext.cc/docs/en/english-vectors.html
```



In [22]:
! kaggle datasets download -d rtatman/glove-global-vectors-for-word-representation -p /content/pre_trained_vec

 99% 452M/458M [00:10<00:00, 37.8MB/s]
100% 458M/458M [00:10<00:00, 45.2MB/s]


In [23]:
! unzip /content/pre_trained_vec/glove-vectors.zip -d /content/pre_trained_vec


Archive:  /content/pre_trained_vec/glove-vectors.zip
  inflating: /content/pre_trained_vec/glove.6B.100d.txt  
  inflating: /content/pre_trained_vec/glove.6B.200d.txt  
  inflating: /content/pre_trained_vec/glove.6B.50d.txt  


In [0]:
# We'll use 50d vectors
glove = open(r'/content/pre_trained_vec/glove.6B.50d.txt', 'r').read()

In [0]:
embedding_dict = {}
for line in glove.split('\n')[:-1]:
  line = line.split()
  embedding_dict[line[0]] = np.asarray(line[1:], dtype = 'float32')

# l = glove.split('\n')[:-1]
# set([len(l[i].split()[1:]) for i in range(len(l))]) = {50}

In [0]:
# we'll create an embedding matirx which will be fed as an argument to Embedding layer
vocab_size = len(word_index) + 1
embedding_dim = 50
embeddings_matrix = np.zeros((vocab_size, embedding_dim)); # 0th index will be padding
not_found_words = []
for word,index in word_index.items():
  vec = embedding_dict.get(word)         # if word is not present it returns None as value
  if vec is not None:
    embeddings_matrix[index] = vec       # words in corpus not found in glove (including oov_token) will be given 0 vector
  else:
    not_found_words.append(word)

In [62]:
# vocab_size-len(not_found_words)  = 94879
vocab_size

508833

In [63]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length, weights=[embeddings_matrix], trainable=False),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 50)            25441650  
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 5, 50)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                29440     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 25,471,155
Trainable params: 29,505
Non-trainable params: 25,441,650
_________________________________________________________________


In [65]:
num_epochs = 10
history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_test, y_test))

Epoch 1/10
40000/40000 [==============================] - 188s 5ms/step - loss: 0.5741 - accuracy: 0.6940 - val_loss: 0.5505 - val_accuracy: 0.7131
Epoch 2/10
40000/40000 [==============================] - 185s 5ms/step - loss: 0.5470 - accuracy: 0.7162 - val_loss: 0.5415 - val_accuracy: 0.7206
Epoch 3/10
40000/40000 [==============================] - 183s 5ms/step - loss: 0.5365 - accuracy: 0.7244 - val_loss: 0.5335 - val_accuracy: 0.7254
Epoch 4/10
40000/40000 [==============================] - 183s 5ms/step - loss: 0.5304 - accuracy: 0.7289 - val_loss: 0.5361 - val_accuracy: 0.7245
Epoch 5/10
40000/40000 [==============================] - 181s 5ms/step - loss: 0.5256 - accuracy: 0.7322 - val_loss: 0.5293 - val_accuracy: 0.7289
Epoch 6/10
40000/40000 [==============================] - 179s 4ms/step - loss: 0.5224 - accuracy: 0.7346 - val_loss: 0.5272 - val_accuracy: 0.7327
Epoch 7/10
 1620/40000 [>.............................] - ETA: 2:25 - loss: 0.5148 - accuracy: 0.7399

KeyboardInterrupt: ignored

*We can see that accuracy is increasing very slowly, so now we'll drop Dropout layer and use CuDNNLSTM for faster implementation*

In [67]:
# We'll now use wiki-text trained corpus which is 300d

!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip -O /content/pre_trained_vec/wiki-text.zip

--2020-05-14 09:29:14--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 2606:4700:10::6816:4b8e, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘/content/pre_trained_vec/wiki-text.zip’

/content/pre_traine 100%[===================>] 650.22M  13.6MB/s    in 49s     

2020-05-14 09:30:04 (13.3 MB/s) - ‘/content/pre_trained_vec/wiki-text.zip’ saved [681808098/681808098]



In [69]:
! unzip /content/pre_trained_vec/wiki-text.zip -d /content/pre_trained_vec/wiki-text

Archive:  /content/pre_trained_vec/wiki-text.zip
  inflating: /content/pre_trained_vec/wiki-text/wiki-news-300d-1M.vec  


In [0]:

wiki_text = open(r'/content/pre_trained_vec/wiki-text/wiki-news-300d-1M.vec', 'r').readlines()

In [0]:
embedding_dict = {}
for line in wiki_text[1:]:
  line = line.split()
  embedding_dict[line[0]] = np.asarray(line[1:], dtype = 'float32')

In [0]:
embedding_dim = 300
embeddings_matrix = np.zeros((vocab_size, embedding_dim)); # 0th index will be padding
not_found_words = []
for word,index in word_index.items():
  vec = embedding_dict.get(word)         # if word is not present it returns None as value
  if vec is not None:
    embeddings_matrix[index] = vec       # words in corpus not found in glove (including oov_token) will be given 0 vector
  else:
    not_found_words.append(word)

In [17]:
print(vocab_size-len(not_found_words))
print(vocab_size)

79590
508833


In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length, weights=[embeddings_matrix], trainable=False),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(64)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 300)           152649900 
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 5, 300)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               187392    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 152,837,421
Trainable params: 187,521
Non-trainable params: 152,649,900
_________________________________________________________________


In [19]:
num_epochs = 10
history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_test, y_test))

Epoch 1/10
40000/40000 [==============================] - 240s 6ms/step - loss: 0.4891 - accuracy: 0.7600 - val_loss: 0.4598 - val_accuracy: 0.7806
Epoch 2/10
40000/40000 [==============================] - 241s 6ms/step - loss: 0.4516 - accuracy: 0.7849 - val_loss: 0.4455 - val_accuracy: 0.7889
Epoch 3/10
40000/40000 [==============================] - 244s 6ms/step - loss: 0.4361 - accuracy: 0.7944 - val_loss: 0.4347 - val_accuracy: 0.7950
Epoch 4/10
40000/40000 [==============================] - 244s 6ms/step - loss: 0.4253 - accuracy: 0.8007 - val_loss: 0.4376 - val_accuracy: 0.7949
Epoch 5/10
  781/40000 [..............................] - ETA: 3:24 - loss: 0.4195 - accuracy: 0.8033

KeyboardInterrupt: ignored